In [1]:
# Ticket 7099 issues identified by clinic team
import json
import os
import datetime, pytz
import pandas as pd
from cacheops import invalidate_model
from copy import copy
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
from django.db.models.deletion import ProtectedError
from edc_appointment.models import Appointment as CaregiverAppointment
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
def get_onschedule_model(subject_identifier, schedule_name):
    try:
        ssh = SubjectScheduleHistory.objects.get(
            subject_identifier=subject_identifier, schedule_name=schedule_name)
    except SubjectScheduleHistory.DoesNotExist:
        return None
    else:
        return getattr(ssh, 'onschedule_model', None)

In [ ]:
tz = pytz.timezone('Africa/Gaborone')
def update_report_datetime(onschedule_cls, report_datetime, pid):
    schedule_name = onschedule_cls.objects.get(subject_identifier=pid).schedule_name
    visit = MaternalVisit.objects.get(subject_identifier=pid, schedule_name=schedule_name, visit_code='2000M')

    report_datetime = tz.localize(report_datetime)
    visit.report_datetime = report_datetime
    visit.save()

In [ ]:
# Correct window period for PID: `B142-040990096-5-70`
consent = CaregiverChildConsent.objects.filter(
    subject_identifier='B142-040990096-5-70').earliest('consent_datetime')
onsch = OnScheduleCohortAEnrollment.objects.get(subject_identifier='B142-040990096-5')
ssh = SubjectScheduleHistory.objects.get(
    subject_identifier=onsch.subject_identifier, schedule_name=onsch.schedule_name)
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=ssh.onschedule_model, name=onsch.schedule_name)
onsch.delete()
ssh.delete()

schedule.put_on_schedule(
    subject_identifier=onsch.subject_identifier,
    onschedule_datetime=consent.created.replace(microsecond=0),
    schedule_name=onsch.schedule_name,
    base_appt_datetime=consent.created.replace(microsecond=0))

onsch = OnScheduleCohortAEnrollment.objects.get(subject_identifier='B142-040990096-5', )
onsch.child_subject_identifier = consent.subject_identifier
onsch.save()

In [ ]:
# Update visit report datetime to correct window periods
# `B142-040990096-5-25`
pid = 'B142-040990096-5'
b_report_datetime = datetime.datetime(2021, 6, 16, 8, 27, 35)
update_report_datetime(OnScheduleCohortBEnrollment, b_report_datetime, pid)

# `B142-040990096-5-70`
a_report_datetime = datetime.datetime(2022, 2, 10, 9, 25, 17)
update_report_datetime(OnScheduleCohortAEnrollment, a_report_datetime, pid)

In [3]:
# Re-save 3000M visit, to correct window period
pid = 'B142-040990360-5'
try:
    visit_3000M = MaternalVisit.objects.get(subject_identifier=pid, visit_code='3000M', visit_code_sequence=0)
except MaternalVisit.DoesNotExist:
    pass
else:
    appt_3001M = CaregiverAppointment.objects.get(subject_identifier=pid, visit_code='3001M')
    schedule_name = appt_3001M.schedule_name
    onschedule_model = get_onschedule_model(pid, schedule_name=schedule_name)
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=onschedule_model, name=schedule_name)
    schedule.put_on_schedule(
            subject_identifier=pid,
            onschedule_datetime=visit_3000M.report_datetime.replace(microsecond=0),
            schedule_name=schedule_name,
            base_appt_datetime=visit_3000M.report_datetime.replace(microsecond=0))

In [4]:
# Remove 2000M visit appt_status `new` PID: B142-040990375-3, B142-040990360-5, B142-040990705-1,
# B142-040990713-5
pids = ['B142-040990375-3', 'B142-040990360-5', 'B142-040990705-1',
        'B142-040990713-5', 'B142-040990804-2']
for pid in pids:
    new_appt_2000M = CaregiverAppointment.objects.filter(
        subject_identifier=pid, appt_status='new', visit_code='2000M')

    for appt in new_appt_2000M:
        schedule_name = appt.schedule_name
        try:
            appt.delete()
        except ProtectedError:
            continue
        else:
            try:
                ssh = SubjectScheduleHistory.objects.get(
                    subject_identifier=pid, schedule_name=schedule_name)
            except SubjectScheduleHistory.DoesNotExist:
                continue
            else:
                onschedule_model_cls = django_apps.get_model(ssh.onschedule_model)
                try:
                    onschedule_obj = onschedule_model_cls.objects.get(
                        subject_identifier=pid, schedule_name=schedule_name)
                except onschedule_model_cls.DoesNotExist:
                    pass
                else:
                    onschedule_obj.delete()
                    ssh.delete()


In [ ]:
# Correct window period for PID: `B142-040990528-7-60`
v2_1_consent = CaregiverChildConsent.objects.get(subject_identifier='B142-040990528-7-60', version='2.1')
v2_consent_dt = datetime.datetime(2022, 9, 21, 11, 32)
v2_consent_dt = tz.localize(v2_consent_dt)
consent_v2 = copy(v2_1_consent)
consent_v2.id = None
consent_v2.version = '2'
consent_v2.consent_datetime = v2_consent_dt
consent_v2.save()
invalidate_model(CaregiverChildConsent)

consent = CaregiverChildConsent.objects.filter(
    subject_identifier='B142-040990528-7-60').earliest('consent_datetime')
onsch = OnScheduleChildCohortBSec.objects.get(subject_identifier='B142-040990528-7-60')
ssh = SubjectScheduleHistory.objects.get(
    subject_identifier=onsch.subject_identifier, schedule_name=onsch.schedule_name)
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=ssh.onschedule_model, name=onsch.schedule_name)
onsch.delete()
ssh.delete()
schedule.put_on_schedule(
    subject_identifier=onsch.subject_identifier,
    onschedule_datetime=consent.created.replace(microsecond=0),
    schedule_name=onsch.schedule_name,
    base_appt_datetime=consent.created.replace(microsecond=0))

In [ ]:
# Resave enrollment visit for C142-040990749-9 siblings to correct window period
vi